In [177]:
import pandas as pd
import numpy as np
from gurobipy import GRB, Model, quicksum

import utils.util as utils

In [178]:
cities_df = pd.read_csv('data/cities_mainline.csv')
# for i, row in cities_df.iterrows():
#     if row['city'] == ''
#     cities_df.at[i, 'has_switching_station'] = row['city']
cities_df['switching_stations'] = 0
cities_df.loc[cities_df['City'] == 'Opole', 'switching_stations'] = 1
cities_df.loc[cities_df['City'] == 'Warszawa', 'switching_stations'] = 2
cities_df.loc[cities_df['City'] == 'Kielce', 'switching_stations'] = 3
cities_df

,Area,City,x,y,TB/s,in_mainline,switching_stations
0,1,Warszawa,100,65,-,1,2
1,1,Skierniewice,92,62,197,1,0
2,1,Lódz,85,58,317,0,0
3,1,Konin,68,64,236,0,0
4,1,Sieradz,72,55,178,0,0
5,1,Piotrków,89,51,139,1,0
6,1,Radom,102,50,288,1,0
7,2,Kielce,98,42,-,1,3
8,2,Czestochowa,79,40,278,0,0
9,2,Bytom,76,31,163,0,0


### Parameters

In [179]:
S = cities_df[cities_df['switching_stations'] > 0]['City'].tolist()
C_not = cities_df[cities_df['in_mainline'] == 0]['City'].tolist()

M = {s: cities_df[cities_df['City'] == s]['switching_stations'].values[0]*400 for s in S}
R = {c_not: int(cities_df[cities_df['City'] == c_not]['TB/s'].values[0]) for c_not in C_not}
d = {(s, c_not): utils.euclidean((cities_df[cities_df['City'] == s]['x'].values[0], cities_df[cities_df['City'] == s]['y'].values[0]), (cities_df[cities_df['City'] == c_not]['x'].values[0], cities_df[cities_df['City'] == c_not]['y'].values[0])) for s in S for c_not in C_not}
P = 1000
L = 200
T = 0.4

In [180]:
model = Model('Capacity optimization')

### Variables

In [181]:
f = model.addVars([(s, c_not) for s in S for c_not in C_not], vtype=GRB.CONTINUOUS, name='f')

### Objective function

In [182]:
model.setObjective(quicksum(P*d[(s, c_not)]*f[s, c_not] for s in S for c_not in C_not), GRB.MINIMIZE)

### Constraints

#### Copper cable constraint, degradation constraint, and non negativity constraint

In [183]:
for s in S:
    for c_not in C_not:
        model.addConstr(f[s, c_not] <= L, f'copper_cable_capacity_{s}_{c_not}')             # Copper cable capacity constraint
        model.addConstr(f[s, c_not] <= 2*np.pi*d[s, c_not], f'degradation_{s}_{c_not}')     # Degradation constraint
        model.addConstr(f[s, c_not] >= 0, f'non_negativity_{s}_{c_not}')                    # Non negativity constraint

#### City requirement constraint

In [184]:
for c_not in C_not:
    # print(R)
    # print(f[s, c_not] for c_not in C_not)
    # print(quicksum(f[s, c_not] for c_not in C_not).getValue())
    model.addConstr(quicksum(f[s, c_not] for s in S) >= R[c_not], f'city_requirement_{c_not}')

#### Maximum capacity constraint

In [185]:
for s in S:
    model.addConstr(quicksum(f[s, c_not] for c_not in C_not) <= M[s], f'maximum_capacity_{s}')

UFO constraint

In [186]:
for s in S:
    model.addConstr(f[s, 'Lódz'] <= T * R['Lódz'], f'UFO_{s}')

### Optimize

In [187]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 126 rows, 36 columns and 183 nonzeros
Model fingerprint: 0x1b6acff4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 6e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+03]
Presolve removed 111 rows and 0 columns
Presolve time: 0.01s
Presolved: 15 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7009370e+06   2.493404e+02   0.000000e+00      0s
      21    6.9349644e+07   0.000000e+00   0.000000e+00      0s

Solved in 21 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.934964446e+07


In [188]:
if model.status == GRB.OPTIMAL:
    model.printStats()
    print()
    
    # What will be the costs of connecting the remaining cities to the switching stations?
    objective_value = model.getObjective().getValue()
    print(f'Cost of connecting the remaining cities to the switching stations: {objective_value:.2f} EUR\n\n')
    
    # What capacity should be installed between Poznań and Wałbrzych and the variou switching stations?
    print('Poznań capacity:')
    for s in S:
        capacity_1 = f[s, 'Poznań'].X
        print(f'- Capacity between {s} and Poznań: {capacity_1:.2f}')
    print('\nWałbrzych capacity:')
    for s in S:
        capacity_2 = f[s, 'Wałbrzych'].X
        print(f'- Capacity between {s} and Wałbrzych: {capacity_2:.2f}')
        
    # How much of the total capacity at the switching station in Kielce is utilized?
    print('\n\nKielce capacity:')
    capacity_3 = sum(f['Kielce', c_not].X for c_not in C_not)
    print(f'- Total capacity at the switching station in Kielce: {M["Kielce"]:.2f}')
    print(f'- Utilized capacity at the switching station in Kielce: {capacity_3:.2f}')
    print(f'- Utilization: {(capacity_3/M["Kielce"])*100:.2f}%, Remaining capacity: {M["Kielce"]-capacity_3:.2f}')


Statistics for model Capacity optimization:
  Linear constraint matrix    : 126 Constrs, 36 Vars, 183 NZs
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 10816.7, 61684.7 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 67.963, 1200 ]

Cost of connecting the remaining cities to the switching stations: 69349644.46 EUR


Poznań capacity:
- Capacity between Warszawa and Poznań: 200.00
- Capacity between Kielce and Poznań: 51.00
- Capacity between Opole and Poznań: 0.00

Wałbrzych capacity:
- Capacity between Warszawa and Wałbrzych: 0.00
- Capacity between Kielce and Wałbrzych: 0.00
- Capacity between Opole and Wałbrzych: 129.00


Kielce capacity:
- Total capacity at the switching station in Kielce: 1200.00
- Utilized capacity at the switching station in Kielce: 1131.00
- Utilization: 94.25%, Remaining capacity: 69.00


#### Verify connections and satisfied requirements

In [189]:
if model.status == GRB.OPTIMAL:
    
    results = {}
    results['City'] = C_not
    for s in S:
        results[f'{s}'] = [1 if f[s, c_not].X > 0 else 0 for c_not in C_not]
    for s in S:
        results[f'{s}_flow'] = [round(f[s, c_not].X, 2) for c_not in C_not]
    
    results['Flow_sum'] = [sum([f[s, c_not].X for s in S]) for c_not in C_not]
    results['Requirement'] = [R[c_not] for c_not in C_not]
    results['Difference'] = [R[c_not] - sum([f[s, c_not].X for s in S]) for c_not in C_not]
    
    results_df = pd.DataFrame(results)
    print(results_df)

                City  Warszawa  Kielce  Opole  Warszawa_flow  Kielce_flow  \
0               Lódz         1       1      1         104.01       126.80   
1              Konin         1       1      0         200.00        36.00   
2            Sieradz         0       1      0           0.00       178.00   
3        Czestochowa         1       1      0         157.96       120.04   
4              Bytom         0       1      1           0.00       154.55   
5          Sosnowiec         1       1      0           3.49       139.51   
6           Katowice         1       1      1          20.31       153.39   
7   Wodzisław Śląski         0       1      0           0.00       119.00   
8             Poznań         1       1      0         200.00        51.00   
9       Zielona Góra         1       1      0         114.24         6.76   
10           Wrocław         0       1      1           0.00        45.95   
11         Wałbrzych         0       0      1           0.00         0.00   

#### Verify switching stations capacities

In [190]:
if model.status == GRB.OPTIMAL:
    
    results = {}
    results['Switching_station'] = S
    results['Maximum_capacity'] = [M[s] for s in S]
    results['Capacity_used'] = [sum([f[s, c_not].X for c_not in C_not]) for s in S]
    results['Remaining_capacity'] = [round(M[s] - sum([f[s, c_not].X for c_not in C_not]), 1) for s in S]
    
    results_df = pd.DataFrame(results)
    print(results_df)

  Switching_station  Maximum_capacity  Capacity_used  Remaining_capacity
0          Warszawa               800          800.0                 0.0
1            Kielce              1200         1131.0                69.0
2             Opole               400          400.0                 0.0


### Sensitivity Analysis

#### Changes in demand

In [191]:
if model.status == GRB.OPTIMAL:
    
    # Get shadow price and RHS bounds
    shadow_price = model.getConstrByName(f'city_requirement_Konin').Pi
    rhs = model.getConstrByName(f'city_requirement_Konin').RHS
    rhs_lower = model.getConstrByName(f'city_requirement_Konin').SARHSLow
    rhs_upper = model.getConstrByName(f'city_requirement_Konin').SARHSUp
    
    print('### Constraint: city_requirement_Konin ###')
    print(f'Shadow price: {shadow_price:.2f}')
    print(f'Right hand side: {rhs:.2f}')
    print(f'Right hand side lower and upper bound: [{rhs_lower:.2f}, {rhs_upper:.2f}]')
    
    print()
    
    increase_rhs_10_percent = rhs * 1.1
    increase_rhs_20_percent = rhs * 1.2
    decrease_rhs_10_percent = rhs * 0.9
    decrease_rhs_20_percent = rhs * 0.8
    
    # Test if any are outside the bounds
    for increased_rhs in [increase_rhs_10_percent, increase_rhs_20_percent]:
        percentage_increase = round(((increased_rhs - rhs) / rhs) * 100, 0)
        if increased_rhs > rhs_upper:
            print(f'Increasing right hand side with {percentage_increase}% ({increased_rhs:.2f}) is OUTSIDE the upper bound {rhs_upper:.2f}')
        else:
            change_in_total_cost = shadow_price * np.abs(increased_rhs - rhs)
            print(f'Increasing right hand side with {percentage_increase}% ({increased_rhs:.2f}) is WITHIN the upper bound {rhs_upper:.2f}, Change in total cost: {change_in_total_cost:.2f} EUR')
    
    # Test if any are outside the bounds
    for decreased_rhs in [decrease_rhs_10_percent, decrease_rhs_20_percent]:
        percentage_increase = round(((decreased_rhs - rhs) / rhs) * 100, 0)
        if decreased_rhs < rhs_lower:
            print(f'Decreasing right hand side with {percentage_increase}% ({decreased_rhs:.2f}) is OUTSIDE the lower bound {rhs_lower:.2f}')
        else:
            change_in_total_cost = shadow_price * np.abs(decreased_rhs - rhs)
            print(f'Decreasing right hand side with {percentage_increase}% ({decreased_rhs:.2f}) is WITHIN the lower bound {rhs_lower:.2f}, Change in total cost: {change_in_total_cost:.2f} EUR')      

### Constraint: city_requirement_Konin ###
Shadow price: 37202.15
Right hand side: 236.00
Right hand side lower and upper bound: [200.00, 305.00]

Increasing right hand side with 10.0% (259.60) is WITHIN the upper bound 305.00, Change in total cost: 877970.75 EUR
Increasing right hand side with 20.0% (283.20) is WITHIN the upper bound 305.00, Change in total cost: 1755941.50 EUR
Decreasing right hand side with -10.0% (212.40) is WITHIN the lower bound 200.00, Change in total cost: 877970.75 EUR
Decreasing right hand side with -20.0% (188.80) is OUTSIDE the lower bound 200.00


In [192]:
# Recreate the model
new_model = Model('Capacity optimization with updated constraint city_requirement_Konin')
f = new_model.addVars([(s, c_not) for s in S for c_not in C_not], vtype=GRB.CONTINUOUS, name='f')
new_model.setObjective(quicksum(P*d[(s, c_not)]*f[s, c_not] for s in S for c_not in C_not), GRB.MINIMIZE)

# Add the constraints
for s in S:
    for c_not in C_not:
        new_model.addConstr(f[s, c_not] <= L, f'copper_cable_capacity_{s}_{c_not}')             # Copper cable capacity constraint
        new_model.addConstr(f[s, c_not] <= 2*np.pi*d[s, c_not], f'degradation_{s}_{c_not}')     # Degradation constraint
        new_model.addConstr(f[s, c_not] >= 0, f'non_negativity_{s}_{c_not}')                    # Non negativity constraint

for s in S:
    new_model.addConstr(quicksum(f[s, c_not] for c_not in C_not) <= M[s], f'maximum_capacity_{s}')  # Maximum capacity constraint

for s in S:
    new_model.addConstr(f[s, 'Lódz'] <= 0.4 * R['Lódz'], f'UFO_{s}')                            # UFO constraint

for c_not in C_not:
    if c_not == 'Konin':
        new_model.addConstr(quicksum(f[s, c_not] for s in S) >= 0.8 * R[c_not], f'city_requirement_{c_not}')  # Updated constraint
    else:
        new_model.addConstr(quicksum(f[s, c_not] for s in S) >= R[c_not], f'city_requirement_{c_not}')  # City requirement constraint
        
# Optimize the model
new_model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 126 rows, 36 columns and 183 nonzeros
Model fingerprint: 0x4b1146f3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 6e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+03]
Presolve removed 111 rows and 0 columns
Presolve time: 0.01s
Presolved: 15 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7009370e+06   2.434404e+02   0.000000e+00      0s
      20    6.7648527e+07   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.764852745e+07


In [193]:
if new_model.status == GRB.OPTIMAL:
    if model.status == GRB.OPTIMAL:
        old_objective_value = model.getObjective().getValue()
        new_objective_value = new_model.getObjective().getValue()
        sign_string = 'lower' if new_objective_value < old_objective_value else 'higher'
        result_string = 'an improvement' if new_objective_value < old_objective_value else 'a worsening'
        
        print(f'No changes: Cost of connecting the remaining cities to the switching stations: {old_objective_value:.2f} EUR')
        print(f'20% decrease: Cost of connecting the remaining cities to the switching stations: {new_objective_value:.2f} EUR\n')
        print(f'A 20% decrease in demand for Konin, causes a {sign_string} total cost, meaning {result_string} of: {old_objective_value - new_objective_value:.2f} EUR')

No changes: Cost of connecting the remaining cities to the switching stations: 69349644.46 EUR
20% decrease: Cost of connecting the remaining cities to the switching stations: 67648527.45 EUR

A 20% decrease in demand for Konin, causes a lower total cost, meaning an improvement of: 1701117.01 EUR


#### Precision of Pi

In [194]:
if model.status == GRB.OPTIMAL:
    
    # Find max lower bound and min upper bound of pi
    max_pi_lower_bound = -np.inf
    min_pi_upper_bound = np.inf
    for s in S:
        for c_not in C_not:
            adjustment_factor = (1 / (2 * d[s, c_not]))
            degradation = model.getConstrByName(f'degradation_{s}_{c_not}')
            pi_lower_bound = degradation.SARHSLow * adjustment_factor
            pi_upper_bound = degradation.SARHSUp * adjustment_factor
            if pi_lower_bound > max_pi_lower_bound:
                max_pi_lower_bound = pi_lower_bound
            if pi_upper_bound < min_pi_upper_bound:
                min_pi_upper_bound = pi_upper_bound
    
    # Calculate the percentage change
    percentage_change_lower = round(((max_pi_lower_bound - np.pi) / np.pi) * 100, 2)
    percentage_change_upper = round(((min_pi_upper_bound - np.pi) / np.pi) * 100, 2)
    
    # Print the results
    print(f'Lower and upper bound of pi for the current basis to remain optimal: {[max_pi_lower_bound, min_pi_upper_bound]} (percentage change: [{percentage_change_lower}%, {percentage_change_upper}%])')

Lower and upper bound of pi for the current basis to remain optimal: [3.123475237772121, 3.220198046143971] (percentage change: [-0.58%, 2.5%])
